in this analysis, we take into controls from ukbiobank

# Read in Data

In [1]:
path_base = "/no-backup/lianyun/20201023_MDD_GenePathwayAssociation_Lucia"
path_lucia = paste0(path_base,"/Data/Data_Lucia")
path_lh = paste0(path_base,"/Data/Data_LH")

wdir = paste0(path_base,"/1_LifetimeMDD/12_tscore_DLPC_CMC_IncludingControl")
setwd(wdir)

source("/projects/biobank/release2/pheno/src/utilities.R")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [10]:
path_pheno = paste0(wdir,"/1_Phenos_tscore_DLPC_CMC_IncludingControl")
path_gwasresults = paste0(wdir,"/2_GWASResults_tscore_DLPC_CMC_IncludingControl")
path_gwassumstats = paste0(wdir,"/3_GWASSumStats_tscore_DLPC_CMC_IncludingControl")
path_manhattanqq = paste0(wdir,"/4_ManhattanQQ_tscore_DLPC_CMC_IncludingControl")
path_locuszoom = paste0(wdir,"/5_Locuszoom_tscore_DLPC_CMC_IncludingControl")
path_topsnps = paste0(wdir,"/6_TopSNPsinAllPheno_tscore_DLPC_CMC_IncludingControl")
path_ldsc = paste0(wdir,"/7_LDSC_tscore_DLPC_CMC_IncludingControl")
path_gc = paste0(wdir,"/8_GeneticCorrelation_tscore_DLPC_CMC_IncludingControl")

In [2]:
data = read.delim(paste0(path_lh,"/Merged_LifetimeMDD_IncludingControls.txt"))

In [3]:
table(data$tscore_DLPC_CMC,useNA="always")


     1      2      3   <NA> 
  7225   3819   3943 319646 

# Prepare pheno data

- Create columns: 1vsRest, 2vsRest, 3vsRest. Replace all NAs as 0.

In [14]:
data$tscore_DLPC_CMC_1vsRest = data$tscore_DLPC_CMC
data$tscore_DLPC_CMC_1vsRest[data$tscore_DLPC_CMC_1vsRest!=1]=0
data$tscore_DLPC_CMC_1vsRest[is.na(data$tscore_DLPC_CMC_1vsRest)]=0

data$tscore_DLPC_CMC_2vsRest = data$tscore_DLPC_CMC
data$tscore_DLPC_CMC_2vsRest[data$tscore_DLPC_CMC_2vsRest!=2]=0
data$tscore_DLPC_CMC_2vsRest[data$tscore_DLPC_CMC_2vsRest==2]=1
data$tscore_DLPC_CMC_2vsRest[is.na(data$tscore_DLPC_CMC_2vsRest)]=0

data$tscore_DLPC_CMC_3vsRest = data$tscore_DLPC_CMC
data$tscore_DLPC_CMC_3vsRest[data$tscore_DLPC_CMC_3vsRest!=3]=0
data$tscore_DLPC_CMC_3vsRest[data$tscore_DLPC_CMC_3vsRest==3]=1
data$tscore_DLPC_CMC_3vsRest[is.na(data$tscore_DLPC_CMC_3vsRest)]=0

- Generate pheno tables

In [21]:
clusters = c("tscore_DLPC_CMC_1vsRest","tscore_DLPC_CMC_2vsRest","tscore_DLPC_CMC_3vsRest")
for (c in clusters){
    n = substr(c,17,17)
    print(n)
}

[1] "1"
[1] "2"
[1] "3"


In [24]:
clusters = c("tscore_DLPC_CMC_1vsRest","tscore_DLPC_CMC_2vsRest","tscore_DLPC_CMC_3vsRest")
for (c in clusters){
    df = data %>% select(IID,c)
    df$IID2 = df$IID
    col_order = c("IID","IID",c)
    df = df[, col_order]
    n = substr(c,17,17)
    write.table(df, file = paste0(path_pheno,"/Pheno_",n,".txt"), sep = "\t", row.names = FALSE, col.names = FALSE)
}

- Generate prevalence tables

In [23]:
Clusters = c("C1vs.Rest","C2vs.Rest","C3vs.Rest")
N_Cases = c(length(which(data$tscore_DLPC_CMC_1vsRest==1)),
            length(which(data$tscore_DLPC_CMC_2vsRest==1)),
            length(which(data$tscore_DLPC_CMC_3vsRest==1)))
N_Controls = c(length(which(data$tscore_DLPC_CMC_1vsRest==0)),
              length(which(data$tscore_DLPC_CMC_2vsRest==0)),
              length(which(data$tscore_DLPC_CMC_3vsRest==0)))

df = data.frame(Clusters,N_Cases,N_Controls)
df$N_sum = rowSums(df[,2:3])
df$Prevalence = df$N_Cases/df$N_sum

print(df)

write.table(df,paste0(path_pheno,"/Prevalence_tscore_DLPC_CMC_IncludingControls.txt"),sep="\t", row.names = FALSE, quote = FALSE)

   Clusters N_Cases N_Controls  N_sum Prevalence
1 C1vs.Rest    7225     327408 334633 0.02159082
2 C2vs.Rest    3819     330814 334633 0.01141250
3 C3vs.Rest    3943     330690 334633 0.01178306


# Run GWAS (in bash)

In [ ]:
# in bash
wdir="/no-backup/lianyun/20201023_MDD_GenePathwayAssociation_Lucia/1_LifetimeMDD/12_tscore_DLPC_CMC_IncludingControl"
genodir="/projects/biobank/release3/genotypes"
phenodir="$wdir/1_Phenos_tscore_DLPC_CMC_IncludingControl"
phenos=$(echo "1" "2" "3")
chromosomes=$(seq 1 22)
covarfile="/no-backup/lianyun/biobank/symptomsgwas/pheno/allchr.unrelatedbritishqced.mafhwe.prunedld1.flashpca.20pcs.arrays.covar" ## ID ID PCS1-20, ARRAY 

for p in $phenos
do echo $p
for chr in $chromosomes 
do echo $chr
bfile="$genodir/ukb_imp_v3_chr$chr.unrelatedbritishqced.maf001geno9"
echo -e "$wdir/2_GWASResults_tscore_DLPC_CMC_IncludingControl/chr$chr.cluster$p.covararraypcs.log: $pheno\n\t\
/home/hpc/lianyun.huang/softwares/plink/plink \
--bfile $bfile --pheno $phenodir/Pheno_$p.txt --logistic hide-covar --ci 0.95 --1 \
--covar $covarfile --allow-no-sex --memory 18000 \
--out $wdir/2_GWASResults_tscore_DLPC_CMC_IncludingControl/chr$chr.cluster$p.covararraypcs; \
touch $wdir/2_GWASResults_tscore_DLPC_CMC_IncludingControl/chr$chr.cluster$p.covararraypcs.log">>$wdir/runGWAS
echo -e "$wdir/2_GWASResults_tscore_DLPC_CMC_IncludingControl/chr$chr.cluster$p.covararraypcs.log">>$wdir/all_runGWAS
done
done

python /home/hpc/lianyun.huang/softwares/make_all_files.py -i $wdir/all_runGWAS -o $wdir/all_runGWAS.sh 
cat $wdir/runGWAS $wdir/all_runGWAS.sh > $wdir/runGWAS.sh 
make --file=$wdir/runGWAS.sh all -d -j 20